In [1]:
from CurriculumVitae import *
from IPython.display import display, HTML, Markdown, clear_output, FileLink
from datetime import date
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights
import os
from mark_status import mark_status

In [2]:
my_resume = Resume(
    person=Person(name="Zach Allen", pronouns="He/Him", 
                    contact_info=ContactInfo(
                       email="fractalmachinist@gmail.com", 
                       phone="+1 (509)438-8146", 
                       link="https://fractalmachini.st",
                       link2="https://linkedin.com/in/zachallen-fractalmachinist/")),

    headline=' '.join(["Machine Learning Researcher & Data Engineer with 5 years experience in collaborative AI innovation & infrastructure implementation at scale.",                       
                       "Expert in Python & TensorFlow since 2015/2017.",
                       "Passionate about working closely with multidisciplinary partners to shape the future towards global sustainability.",
                       "Certified in data-driven integrations of business needs, software engineering, project management, and software development best practices.",
                       "Excellent writing, speaking, presenting, and technical communication skills.",
                       ]))

In [3]:
my_resume.education = [Occupation(
    title="Bachelor's in Computer Science", location="Utah, USA (Online)",
    timespan=Between(start=date(2019, 2, 1), end=date(2022, 8, 31)),
    subtitle="Western Governors University", 
    headline="Focused on Machine Learning and Project Management in a fully remote environment",
    skills=my_resume.Skills("Remote Work", "Project Management", "Machine Learning"),
    sub_tasks=[
        Effort(title="Diamond Price Prediction Model",
               headline="Proposed, implemented, and documented a diamond price prediction ML model",
               website="https://github.com/FractalMachinist/WGU-C968",
               skills=my_resume.Skills("Machine Learning", "Analysis", "Statistics", "Feature Engineering", "End to End"),
               achievements=[
                  Achievement(
                       headline="Demonstrated integration of project management and software engineering best practices.",
                       skills=my_resume.Skills("Project Management", "Stakeholder Support", "Written Communication", "Documentation", "Managing Requirements", "Technical Communication")),
                   Achievement(
                       headline="Developed simple Data Pipeline in python/pandas.",
                       skills=my_resume.Skills("Python", "Data", "Software Engineering", "NumPy", "SciPy", "Pandas")),
                   Achievement(
                       headline="Integrated code and report generation with a Jupyter Notebook.",
                       skills=my_resume.Skills("Jupyter", "Seaborn", "Data Visualization"))
               ]

        ), 
        Effort(title="Advanced Java Concepts",
               headline="Developed appointment scheduling and customer database tool in Java",
               website="https://github.com/FractalMachinist/C195-Scheduling-App",
               skills=my_resume.Skills("Java", "Software Engineering", "Git", "SQL"),
               achievements=[
               Achievement(
                   headline="Encapsulated Observable state and database connection state into a shared wrapper class, streamlining inheritance, error handling, and UI/Database auto-updates.",
                   skills=my_resume.Skills("Java", "JavaFX", "JDBC", "Integration", "API Design", "MySQL")),
               Achievement(
                   headline="Paid technical debt by finalizing project documentation.",
                   skills=my_resume.Skills("Documentation", "Written Communication", "Ownership", "SDLC", "Project Management", "Scope"))
               ]),    
    ],
    
    achievements=[
        
        Certification(name="CompTIA Project+",
                      headline="Demonstrated understanding of Project Management roles, processes, and documentation.",
                      issuer="Pearson VUE", issue_date=date(2019, 7, 24),
                      website="https://wsr.pearsonvue.com/testtaker/authenticate/AuthenticateScoreReport.htm",
                      confirmation_info={"Registration":"358639011", "Validation":"155946649"},
                      skills=my_resume.Skills("Project Management", "Managing Requirements", "SDLC", "Integration", "Agile", "Scope", "Stakeholder Support")),
        
        Certification(name="IT Information Library Foundations Certification (ITIL)", 
                      headline="Demonstrated understanding of designing, deploying, maintaining, and retiring IT resources.",
                      issuer="AXELOS", issue_date=date(2020, 8, 1),
                      skills=my_resume.Skills("Project Management", "Managing Requirements", "SDLC", "Integration")),
        
        Certification(name="Site Development Associate",
                      headline="Demonstrated ability to design and build websites.",
                      issuer="CIW", issue_date=date(2019, 2, 1),
                      skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript",)),
        
        Achievement(headline="Excellence Award for Communication Applications",
                    portfolio_link="https://github.com/FractalMachinist/Rust_Business_Presentation",
                    skills=my_resume.Skills("Written Communication", "Technical Communication")),
        
    ])]

In [4]:
my_resume.employment = [
    Occupation(title="Sofware Engineer", location="St. Louis, Missouri, USA", website="https://PlutonBio.com",
               timespan=Between(start=date(2021, 3, 1), end=date(2021, 8, 31)),
               subtitle="Pluton Biosciences", supervisor=Person(name="Dr. Boahemaa Adu-Oppong", pronouns="She/Her",
                                                                contact_info=ContactInfo(email="BAdu-Oppong@plutonbio.com")),
               headline="Data Engineering supporting Bioinformatics Research",
               skills=my_resume.Skills("Remote Work", "Big Data", "Bioinformatics", "Genetics", "Multidisciplinary", "Motivated"),
               sub_tasks=[
                   Effort(
                       title="AWS Genomics Pipeline",
                       headline="Independently designed, implemented, and deployed Terabyte-scale AWS genomics pipeline in 4 months part time, from no prior genomics or AWS experience.",
                       skills=my_resume.Skills("AWS", "Cloud Computing", "Data", "Big Data", "Large Scale", "Infrastructure", "End to End", "Motivated", "Multidisciplinary", "Problem Solving", "Remote Work", "Innovation", "Curious", "Ownership", "Project Management", "SDLC", "Data Architecture", "Data Infrastructure"),
                       achievements=[
                           Achievement(
                               # title="Pipeline Design",
                               headline="Designed, implemented, and documented gRPC+ProtoBuf pipeline architecture.",
                               skills=my_resume.Skills("Infrastructure", "Data", "Big Data", "Large Scale", "Tooling", "Integration", "Automation", "Prototyping", "Workflows", "Innovation", "Problem Solving"),
                           ),
                           Achievement(
                               # title="User-Friendly Query Library",
                               # headline="Designed, implemented, developed training for, and documented an in-house Python+R library and interface enabling (non-CS) biologists to easily deploy multi-stage genomics queries without relying on the CS team.",
                               headline="Accelerated research iteration with an in-house Python+R interface helping non-CS teammates deploy complex, multi-stage genomics queries without relying on the CS team.",
                               skills=my_resume.Skills("Python", "R", "API Design", "Libraries", "Tooling", "Automation", "Workflows", "Problem Solving", "Stakeholder Support")
                           ),
                           Achievement(
                               # title="Aligned Deliverables with Stakeholder Needs",
                               headline="Collaborated with SMEs to gather, interpret, and meet data pipeline requirements. Interfaced with Bioinformatics expert for genomics tool selection.",
                               skills=my_resume.Skills("Managing Requirements", "Written Communication", "Project Management", "Collaboration", "Multidisciplinary", "Scope", "Stakeholder Support", "Technical Communication")
                           ),
                       ],
                   ),
                       
                   Effort(
                       title="Containerized Deployment & Autoscaling",
                       headline="Researched, containerized, documented, and deployed 10+ Computational Genomics tools to AWS ECS and EC2.",
                       skills=my_resume.Skills("Integration", "Bash", "Linux", "AWS", "Containerization", "Large Scale", "Frameworks", "Infrastructure", "Bioinformatics", "Genetics", "ECS", "EC2"),
                       achievements=[
                           Achievement(
                               # title="Streamlined Pipeline Development Lifecycle",
                               # headline="Developed, documented, and utilized frameworks for automating Computational Genomics tool containerization.",
                               headline="Streamlined & automated containerization of Computational Genomics tools.",
                               skills=my_resume.Skills("Frameworks", "Infrastructure", "Tooling", "SDLC", "Automation", "Workflows", "Innovation", "Problem Solving")
                           ),
                           Achievement(
                               headline="Constructed and tuned EC2&ECS autoscaling, maximizing cost efficiency and cluster responsiveness.",
                               skills=my_resume.Skills("AWS", "Infrastructure", "Budget", "Monitoring", "Tracking", "Reporting")
                           ),
                           Achievement(
                               headline="Detected, diagnosed, and resolved compute resource inefficiencies.",
                               skills=my_resume.Skills("Optimization", "Software Engineering", "Computer Science", "Debugging", "Computer Engineering", "Iteration", "Monitoring", "Tracking", "Reporting")
                           ),
                       ]
                   ),    
                   
                   Achievement(
                       # title="Spearheaded SDLC & CI/CD",
                       headline="Led the CS team in adopting SDLC tools like Git, AWS CodeCommit, and Docker / AWS Elastic Container Registry. Championed utilization of Microsoft Teams Kanban tools for project&process management.",
                       skills=my_resume.Skills("Leadership", "SDLC", "Git", "Containerization", "Testing", "Integration", "Frameworks", "Project Management", "Teamwork", "Workflows", "CI/CD", "Organizational Skills")
                   ),
                   Achievement(
                       # title="Microbial Taxonomy Visualizations in R",
                       headline="Worked closely with SMEs to perform analysis and visualization of microbial taxonomy data in R.",
                       skills=my_resume.Skills("R", "Analysis", "Statistics", "Written Communication", "Collaboration", "Multidisciplinary", "Teamwork", "Iteration", "Data Visualization", "Technical Communication")
                   ),
               ],
    ),
    
    Occupation(title="Embedded Systems Engineer", location="Allentown, Pennsylvania, USA", website="https://AppliedSeparations.com",
               timespan=Between(start=date(2019, 1, 1), end=date(2020, 1, 25)),
               subtitle="Applied Separations", supervisor=Person(name="Aaron Allen", pronouns="He/Him", 
                                                                contact_info=ContactInfo(email="mnrnln@gmail.com")),
               
               headline="Developed custom C++ / Arduino pump control software",
               achievements=[
                   Achievement(headline="Designed and developed pump control software for chromatography and analytical chemistry systems in C and C++.",
                               skills=my_resume.Skills("C", "C++", "Software Engineering", "SDLC", "Microcontrollers", "Multidisciplinary")),
                   
                   Achievement(headline="Wrote a simple heuristic scheduler & virtual threading to manage real-time (60Hz+) pump control, touch screen input, and data/control communication, all on a single Arduino Mega.",
                               skills=my_resume.Skills("Algorithms", "Optimization", "Software Engineering", "Statistics", "Computer Science", "Microcontrollers", "Computer Engineering")),
                   
                   Achievement(headline="Designed and validated mass delivery tracking, prediction, and smoothing algorithms, to handle nonlinear feedback delay.",
                               skills=my_resume.Skills("Algorithms", "Algebra", "Software Engineering", "Testing", "Computer Science", "Prototyping", "Analysis", "Monitoring", "Reporting")),
                   
                   Achievement(headline="Interfaced with separate development team in charge of chromatography control software. Collected and implemented requirements from instrumentation engineer.",
                               skills=my_resume.Skills("Written Communication", "Managing Requirements", "Project Management", "SDLC", "Documentation", "Collaboration", "Ownership", "Integration", "Leadership", "Git", "Reporting", "Technical Communication"))]),
              
    Occupation(title="Service Writer", location="Richland, Washington, USA", website="https://AlphaComputerCenter.com",
               timespan=Between(start=date(2017, 12, 1), end=date(2018, 11, 30)),
               subtitle="Alpha Computer Center", supervisor=Person(name="Frank Ward Jr.", pronouns="He/Him",
                                                                   contact_info=ContactInfo(email="frankjr@alphacomputercenter.com",
                                                                   phone="+1 (509)946-4230")),
               
               headline="Customer service, sales, and technician support",
               achievements=[
                   Achievement(headline="Ensured customers were able to accurately understand and communicate with repair technicians, improving customer service and reducing diagnostic time.",
                               skills=my_resume.Skills("Written Communication")),
                   
                   Achievement(headline="Leveraged extensive Linux experience to rapidly identify and repair issues that couldn't be fixed by Mac diagnostic tools.",
                               skills=my_resume.Skills("Linux", "Bash")),
                   
                   Achievement(headline="Reduced call frequency with an informative website. See it on <a href='https://web.archive.org/web/20180113192132/http://www.alphacomputercenter.com/wordpress1/'>web archive</a>.",
                               skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "Graphic Design"))]),
              
    Occupation(title="Machine Learning Researcher (Intern)", location="Richland, Washington, USA", website="https://pnnl.gov",
               timespan=Between(start=date(2017, 1, 1), end=date(2017, 5, 31)),
               subtitle="Pacific Northwest National Laboratory", supervisor=Person(name="Dr. Enoch Yeung", pronouns="He/Him",
                                                                                   contact_info=ContactInfo(email="eyeung@ucsb.edu")),
               
               headline="ML Research and Data Engineering intern",
               achievements=[
                   Achievement(headline="Designed and tested novel Neural Network algorithms, architectures, and error formulations for NLP, image classification, and time-series data classification.",
                               skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Deep Learning", "Neural Network Architectures", "Feature Engineering",
                                                       "Algorithms", "Algebra", "NLP", "Data", "Curious", "Research", "Frameworks", "Computer Science",
                                                       "MatLab", "NumPy", "Software Engineering")),
                   
                   Achievement(headline="Demonstrated increased test accuracy (15% ⇾ 90% detection with higher Bayesian Confidence) on unbalanced (>1000:1) datasets, without duplication, augmentation, or batch filtering.",
                               skills=my_resume.Skills("Machine Learning", "Algebra", "Statistics", "Testing", "Research", "Algorithms", "Computer Science", "Analysis")),
                   
                   Achievement(headline="Worked independently, balancing multiple projects and deliverables with minimal mentor supervision.",
                               skills=my_resume.Skills("Project Management", "Reporting", "Ownership"))])
]

In [5]:
my_resume.projects = [
    Effort(title="Neural Cellular Segmentation",
           headline="Exploring neural cellular automata and attention (NCA+A) for medical image segmentation",
           website="https://github.com/FractalMachinist/NeuralCellularAutomataAttn",
           achievements=[
               Achievement(headline="Developed, tested, and iterated NCA+A models, balancing system resources and model size.",
                           skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Neural Network Architectures", "Research", "Algebra", 
                                                   "Optimization", "Statistics", "NumPy", "Analysis")),
               Achievement(headline="Created multiple tf.Data pipelines with preprocessing and data augmentation steps.",
                           skills=my_resume.Skills("Python", "Data", "Software Engineering", "TensorFlow", "Git", "Frameworks", "Computer Science")),
           ]),
    Effort(title="Interplan",
           headline="Task dependency management from a Graph Database",
           website="https://github.com/FractalMachinist/Interplan",
           achievements=[
               Achievement(headline="Developed a Neo4J+JS dependency resolution and task status tracking API.",
                           skills=my_resume.Skills("JavaScript", "Testing", "Project Management", "Data Driven", "API Design",
                                                   "Software Engineering", "Neo4J", "Apache Tinkerpop")),
               Achievement(headline="Designed intuitive React app for displaying and interacting with tasks.",
                           skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "React")),
               Achievement(headline="Packaged Node+Neo4J in Docker & Kubernetes for easy migration.",
                           skills=my_resume.Skills("Containerization", "Infrastructure"))
           ]),
    
    Effort(title="MarkNotes",
           headline="Intuitive journaling tool designed to encourage long-term review and introspection",
           website="https://github.com/FractalMachinist/MarkNotes",
           achievements=[
               Achievement(headline="Implemented MongoDB and Node API for destructuring, storing, and querying Markdown entries as semi-structured data.",
                           skills=my_resume.Skills("MongoDB", "Data", "Data Driven", "NoSQL", "API Design", "Software Engineering")),
               Achievement(headline="Designed simple, intuitive React web app interface.",
                            skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "Graphic Design")),
               Achievement(headline="Packaged Node+MongoDB in Docker & Kubernetes.",
                           skills=my_resume.Skills("Containerization", "Infrastructure"))
           ]),
    
    # Effort(title="NetTimeLog",
    #        headline="Minimalist, accurate time tracking",
    #        # website="https://fractalmachini.st/demos/nettimelog",
    #        achievements=[
    #            Achievement(headline="Created time-tracking web app with Python+Flask which records what you just completed, so you never estimate what you will do or how long it will take.",
    #                        skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript"))])
]

# Pulling Job Data

In [6]:
jobs = get_jobs()
raw_job_details = get_raw_job_details(jobs)
# job_descriptions = skill_weights.get_job_descriptions(raw_job_details)
job_skill_weights = skill_weights.get_job_skill_weights(raw_job_details)

# Generation

In [7]:
skill_text_weights_con = lambda weights, job_id: weights.loc[job_id, :].groupby(level=0).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [8]:
# my_resume.write_html_to_file(
#     stylesheet='chalkboard',
#     filepath="docs/test/web.html",
#     should_render_all=True
# )

In [9]:
# assert input("Ready to generate? Type 'OK'") == "OK"
# for job_id, job_data in jobs.iterrows():
#     private_path = f"docs/pdf_sources/{job_id}"
#     os.makedirs(private_path, exist_ok=True)
#     my_resume.write_html_to_file(
#         stylesheet='lighttheme',
#         filepath=f"{private_path}/Zach_Allen_Resume.html",
#         alt_template_prefixes={'*':'pdf'},
#         skill_text_weights = skill_text_weights_con(job_skill_weights, job_id)
#     )

In [22]:
for job_id, job_data in jobs.sort_index().iterrows():
    clear_output()
    
    # Construct the PDF
    os.makedirs(f"docs/pdf_sources/{job_id}", exist_ok=True)
    pdf_fpath = f"./docs/pdf_sources/{job_id}/Zach_Allen_Resume.pdf"
    my_resume.export_fitted_pdf(
        pdf_fpath,
        stylesheet='lighttheme',
        alt_template_prefixes={'*':'pdf'},
        skill_text_weights = skill_text_weights_con(job_skill_weights, job_id),
    )
    
    # Display links to job info
    print(job_id)
    print(job_data['company_name'])
    display(HTML(f"<a href='{job_data['url']}'>job</a>"))
    display(FileLink(pdf_fpath))
    
    # Wait for feedback on submission status
    status = input("Status:")
    if status == "skip":
        continue;
    if status == "":
        status = "applied"
    
    mark_status(job_id, status_key_or_id = status)
    

e7276640-0bab-43b7-b635-894fd691b9b5
Tapad


/experiments/FractalMachinist.github.io/docs/pdf_sources/e7276640-0bab-43b7-b635-894fd691b9b5/Zach_Allen_Resume.pdf